## 1) Imports & paths

Load core libraries, point to the project/data locations, and confirm dependency versions plus file availability.


In [ ]:
import warnings
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd

from statsmodels.tsa.statespace.sarimax import SARIMAX  # optional seasonal ARIMA capability
try:
    from prophet import Prophet
    HAVE_PROPHET = True
    PROPHET_IMPORT_ERROR = None
except Exception as exc:  # pragma: no cover - environment dependent
    HAVE_PROPHET = False
    PROPHET_IMPORT_ERROR = exc

warnings.filterwarnings("ignore")

PROJ = Path(r"D:\MGA\job\data analyst\flight-price-analytics")
DATA = PROJ / "data"
FACT = DATA / "fares_fact.csv"
OUT_DETAIL = DATA / "forecast_detail.csv"
OUT_SUM = DATA / "forecast_summary.csv"

print(f"Notebook run at: {datetime.now():%Y-%m-%d %H:%M:%S}")
print(f"Pandas {pd.__version__} | NumPy {np.__version__}")
print(f"Prophet available: {HAVE_PROPHET}")
if not HAVE_PROPHET:
    print(f"Prophet import error: {PROPHET_IMPORT_ERROR}")

for path in [PROJ, DATA, FACT]:
    print(f"{path} exists: {path.exists()}")

if FACT.exists():
    size_mb = FACT.stat().st_size / 1_048_576
    print(f"Fact file size: {size_mb:,.1f} MB")


## 2) Load clean fact & prep

Read the fare fact table, enforce numeric pricing, drop incomplete rows, create the route key, and roll up to a tidy time-series table.


In [ ]:
def load_fares_fact(path: Path) -> pd.DataFrame:
    """Load fares_fact.csv with date parsing and price coercion."""
    date_cols = ["search_date", "depart_date", "snapshot_date"]
    df = pd.read_csv(
        path,
        parse_dates=date_cols,
        dtype={"source_name": "string", "origin": "string", "destination": "string"},
    )
    df["price"] = pd.to_numeric(df["price"], errors="coerce")
    df = df.dropna(subset=["price"] + date_cols)
    df["origin"] = df["origin"].str.strip().str.upper()
    df["destination"] = df["destination"].str.strip().str.upper()
    df["route"] = df["origin"] + "-" + df["destination"]
    return df

def make_timeseries(df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate to a single median price per route-date snapshot."""
    ts = (
        df.groupby(["route", "snapshot_date"], as_index=False)["price"]
        .median()
        .rename(columns={"snapshot_date": "date"})
        .sort_values(["route", "date"])
        .reset_index(drop=True)
    )
    return ts

fact_df = load_fares_fact(FACT)
ts = make_timeseries(fact_df)

print(f"Raw fact rows: {len(fact_df):,} | Routes: {fact_df['route'].nunique():,}")
print(f"Time-series rows: {len(ts):,} spanning {ts['date'].min().date()} to {ts['date'].max().date()}")
print(ts.head())


## 3) Seasonal-naive baseline (weekly p=7)

Create both a simple row-shift and a nearest-neighbor seasonal-naive baseline and keep the most informed value for each route-date.


In [ ]:
def seasonal_naive_shift(ts_df: pd.DataFrame, lag: int = 7, value_col: str = "price", out_col: str = "baseline_7d") -> pd.DataFrame:
    """Shift each route's series by the requested lag to form a fast baseline."""
    work = ts_df.sort_values(["route", "date"]).copy()
    work[out_col] = work.groupby("route", sort=False)[value_col].shift(lag)
    return work

def seasonal_naive_nearest(ts_df: pd.DataFrame, lag_days: int = 7, tolerance: str = "1D", value_col: str = "price", out_col: str = "baseline_7d_nn") -> pd.DataFrame:
    """Use merge_asof to find the closest snapshot near t-7 when exact matches are missing."""
    work = ts_df.sort_values(["route", "date"]).copy()
    history = work[["route", "date", value_col]].copy()
    history["date"] = history["date"] + pd.Timedelta(days=lag_days)
    history = history.rename(columns={value_col: out_col})
    merged = pd.merge_asof(
        work,
        history,
        on="date",
        by="route",
        direction="nearest",
        tolerance=pd.Timedelta(tolerance),
    )
    return merged

ts_shift = seasonal_naive_shift(ts)
ts_nn = seasonal_naive_nearest(ts_shift)
ts_nn["fcst_baseline"] = ts_nn["baseline_7d_nn"].combine_first(ts_nn["baseline_7d"])
ts_features = ts_nn.copy()

coverage = 1 - ts_features["fcst_baseline"].isna().mean()
print(f"Baseline coverage (rows with forecast): {coverage:.1%}")
print(ts_features.head())


## 4) Rolling backtest setup (walk-forward)

Filter to rows with a valid baseline, align actual vs. forecast chronologically, and add rolling diagnostics for later KPIs.


In [ ]:
def backtest_baseline(ts_df: pd.DataFrame, actual_col: str = "price", forecast_col: str = "fcst_baseline") -> pd.DataFrame:
    """Retain rows with baseline forecasts for evaluation."""
    cols = ["route", "date", actual_col, forecast_col]
    bt = ts_df.loc[:, cols].dropna(subset=[forecast_col]).copy()
    bt = bt.rename(columns={actual_col: "actual", forecast_col: "fcst_baseline"})
    bt = bt.sort_values(["route", "date"]).reset_index(drop=True)
    return bt

def add_backtest_features(bt_df: pd.DataFrame, window: str = "30D", min_periods: int = 7) -> pd.DataFrame:
    """Append rolling medians and win-rate helpers per route."""
    enriched = []
    for route, grp in bt_df.groupby("route", sort=False):
        grp = grp.sort_values("date").copy()
        grp["abs_error"] = (grp["actual"] - grp["fcst_baseline"]).abs()
        median_price = grp.set_index("date")["actual"].rolling(window=window, min_periods=min_periods).median()
        grp["median_30d"] = median_price.values
        median_abs_error = grp.set_index("date")["abs_error"].rolling(window=window, min_periods=min_periods).median()
        grp["median_abs_err_30d"] = median_abs_error.values
        grp["win_flag"] = grp["abs_error"] <= grp["median_abs_err_30d"]
        grp["win_flag"] = grp["win_flag"].fillna(False)
        enriched.append(grp)
    return pd.concat(enriched, ignore_index=True)

bt_baseline = backtest_baseline(ts_features)
bt_with_features = add_backtest_features(bt_baseline)

print(f"Backtest rows: {len(bt_with_features):,}")
print(f"Routes modeled: {bt_with_features['route'].nunique():,}")
print(bt_with_features.head())


## 5) KPIs & win-rate summary

Aggregate MAE/MAPE and rolling win-rate per route to understand baseline quality.


In [ ]:
def kpis_by_route(df: pd.DataFrame, actual_col: str = "actual", pred_col: str = "fcst_baseline", win_flag_col: str | None = None) -> pd.DataFrame:
    """Compute MAE, MAPE, counts, and optional win-rate for each route."""
    records: list[dict] = []
    for route, grp in df.groupby("route", sort=False):
        grp_valid = grp.dropna(subset=[actual_col, pred_col])
        if grp_valid.empty:
            continue
        actual = grp_valid[actual_col].astype(float)
        pred = grp_valid[pred_col].astype(float)
        abs_err = (actual - pred).abs()
        mae = abs_err.mean()
        mape = np.nan
        mape_mask = actual > 0
        if mape_mask.any():
            mape = (abs_err[mape_mask] / actual[mape_mask]).mean() * 100
        record = {
            "route": route,
            "mae": mae,
            "mape": mape,
            "n_obs": len(grp_valid),
            "start_date": grp_valid["date"].min(),
            "end_date": grp_valid["date"].max(),
        }
        if win_flag_col and win_flag_col in grp_valid.columns:
            record["win_rate"] = grp_valid[win_flag_col].mean()
        records.append(record)
    result = pd.DataFrame(records)
    if not result.empty:
        result = result.sort_values("mape").reset_index(drop=True)
    return result

baseline_summary = kpis_by_route(bt_with_features, win_flag_col="win_flag")
baseline_summary = baseline_summary.rename(
    columns={
        "mae": "mae_baseline",
        "mape": "mape_baseline",
        "n_obs": "n_obs",
        "win_rate": "win_rate_baseline",
    }
)
if "win_rate_baseline" not in baseline_summary.columns:
    baseline_summary["win_rate_baseline"] = np.nan

print(f"Baseline KPI routes: {len(baseline_summary):,}")
print(baseline_summary.head())


## 6) Prophet per route (optional)

Fit an expanding-window Prophet model when available (N?100) and merge the forecasts for side-by-side evaluation.


In [ ]:
def prophet_walkforward(route_df: pd.DataFrame, min_history: int = 100, refit_every: int = 30) -> pd.Series:
    """Return expanding-window Prophet forecasts aligned to the route's index."""
    work = route_df.sort_values("date").copy()
    work = work.reset_index().rename(columns={"index": "orig_index"})
    preds = np.full(len(work), np.nan, dtype=float)
    if (not HAVE_PROPHET) or len(work) < min_history:
        return pd.Series(preds, index=work["orig_index"])
    model = None
    fitted_until = None
    for i in range(len(work)):
        if i < min_history:
            continue
        need_refit = (model is None) or (fitted_until is None) or ((i - fitted_until) >= refit_every)
        if need_refit:
            train = work.iloc[:i][["date", "price"]].rename(columns={"date": "ds", "price": "y"})
            try:
                model = Prophet(
                    growth="flat",
                    weekly_seasonality=True,
                    daily_seasonality=False,
                    yearly_seasonality=False,
                )
                model.fit(train)
                fitted_until = i
            except Exception as exc:  # pragma: no cover - depends on cmdstan availability
                warnings.warn(f"Prophet fit failed for route {route_df['route'].iloc[0]}: {exc}")
                return pd.Series(preds, index=work["orig_index"])
        future = pd.DataFrame({"ds": [work.loc[i, "date"]]})
        try:
            preds[i] = model.predict(future)["yhat"].iloc[0]
        except Exception as exc:  # pragma: no cover - depends on model stability
            warnings.warn(f"Prophet predict failed for route {route_df['route'].iloc[0]}: {exc}")
            break
    return pd.Series(preds, index=work["orig_index"])

if "fcst_prophet" not in ts_features.columns:
    ts_features["fcst_prophet"] = np.nan

if HAVE_PROPHET:
    for route, idx in ts_features.groupby("route").groups.items():
        route_slice = ts_features.loc[idx, ["route", "date", "price"]]
        preds = prophet_walkforward(route_slice, min_history=100, refit_every=30)
        ts_features.loc[preds.index, "fcst_prophet"] = preds.values
else:
    print("Prophet unavailable in this environment; skipping advanced model.")

if "fcst_prophet" in bt_with_features.columns:
    bt_with_features = bt_with_features.drop(columns=["fcst_prophet"])

bt_with_features = bt_with_features.merge(
    ts_features[["route", "date", "fcst_prophet"]],
    on=["route", "date"],
    how="left",
)

if HAVE_PROPHET and ts_features["fcst_prophet"].notna().any():
    bt_prophet = bt_with_features.dropna(subset=["fcst_prophet"])
    prophet_summary = kpis_by_route(bt_prophet, pred_col="fcst_prophet")
    prophet_summary = prophet_summary.rename(
        columns={
            "mae": "mae_prophet",
            "mape": "mape_prophet",
            "n_obs": "n_obs_prophet",
            "start_date": "start_date_prophet",
            "end_date": "end_date_prophet",
        }
    )
    print(f"Prophet KPI routes: {len(prophet_summary):,}")
    print(prophet_summary.head())
else:
    prophet_summary = pd.DataFrame()
    print("No Prophet KPIs generated (insufficient data or library missing).")


## 7) Save outputs for Power BI

Combine baseline + Prophet metrics, export detail/summary CSVs, and preview the saved tables.


In [ ]:
detail_cols = ["route", "date", "actual", "fcst_baseline", "fcst_prophet", "median_30d"]
forecast_detail = bt_with_features.loc[:, detail_cols].sort_values(["route", "date"])
forecast_detail.to_csv(OUT_DETAIL, index=False)

summary_export = baseline_summary.copy()
if prophet_summary.empty:
    summary_export["mae_prophet"] = np.nan
    summary_export["mape_prophet"] = np.nan
else:
    summary_export = summary_export.merge(
        prophet_summary[["route", "mae_prophet", "mape_prophet"]],
        on="route",
        how="left",
    )
summary_export = summary_export.sort_values("mape_baseline").reset_index(drop=True)
summary_export.to_csv(OUT_SUM, index=False)

print(f"Detail rows saved: {len(forecast_detail):,} -> {OUT_DETAIL}")
print(forecast_detail.head())
print("
Summary snapshot:")
print(summary_export.head())
print(f"Summary rows saved: {len(summary_export):,} -> {OUT_SUM}")
